In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy
import random

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, PReLU, GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical, Sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169
import tensorflow as tf
from skimage.transform import AffineTransform, warp

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics
import tensorflow_hub as hub

from efficientnet.keras import EfficientNetB0
from keras_squeeze_excite_network.se_resnext import SEResNextImageNet as SEResNext

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [4]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [5]:
y = y.set_index("image_id")

In [6]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [7]:
def affine_image(img):
    """

    Args:
        img: (h, w) or (1, h, w)

    Returns:
        img: (h, w)
    """
    # ch, h, w = img.shape
    # img = img / 255.
    if img.ndim == 3:
        img = img[0]

    # --- scale ---
    min_scale = 0.8
    max_scale = 1.2
    sx = np.random.uniform(min_scale, max_scale)
    sy = np.random.uniform(min_scale, max_scale)

    # --- rotation ---
    max_rot_angle = 7
    rot_angle = np.random.uniform(-max_rot_angle, max_rot_angle) * np.pi / 180.

    # --- shear ---
    max_shear_angle = 10
    shear_angle = np.random.uniform(-max_shear_angle, max_shear_angle) * np.pi / 180.

    # --- translation ---
    max_translation = 4
    tx = np.random.randint(-max_translation, max_translation)
    ty = np.random.randint(-max_translation, max_translation)

    tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))
    transformed_image = warp(img, tform)
    assert transformed_image.ndim == 2
    return (transformed_image*255).astype(np.uint8)

In [8]:
size=(120, 120)

def calcRotate(img):
    detector = cv2.ORB_create()
    keypoints=detector.detect(img)
    descriptors=detector.compute(img,keypoints)
    angles=[]
    weights=[]
    for i in descriptors[0]:
        if i.angle!=-1:
            angles.append(i.angle)
            weights.append(i.response)
    if len(angles)==0:
        return 0
    else:
        return np.average(angles,weights=weights)

def upper(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[up:mid,:])==0:
            up=mid
        else:
            bottom=mid
    return max(up-mergin,0)

def lower(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[mid:bottom,:])==0:
            bottom=mid
        else:
            up=mid
    return min(bottom+mergin,img.shape[0])

def lefter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,left:mid])==0:
            left=mid
        else:
            right=mid
    return max(left-mergin,0)

def righter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,mid:right])==0:
            right=mid
        else:
            left=mid
    return min(right+mergin,img.shape[1])

def transformImg(img,size=(255,255),training=True):
    ret2, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    img = 255-img
    if training:
        img=affine_image(img)
    img = img[upper(img):lower(img),lefter(img):righter(img)]
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img, (size[0],size[1]))
    kernel = np.ones((3,3),np.float32)/9
    img = cv2.filter2D(img,-1,kernel)
    return img

def randomErase(img, prob=True):
    # random erasing
    # https://github.com/yu4u/cutout-random-erasing
    p = 0.5
    s_l = 0.02
    s_h = 0.4
    r_1 = 0.3
    r_2 = 1 / 0.3
    v_l = 0
    v_h = 255
    input_size=size[0]
    if prob==False or np.random.random()<p:

        while True:
            s = np.random.uniform(s_l, s_h) * input_size * input_size
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, input_size)
            top = np.random.randint(0, input_size)
            if left + w <= input_size and top + h <= input_size:
                break
        c = np.random.uniform(v_l, v_h, (h, w, 3))
        img[top : top + h, left : left + w, :] = c
    return img

In [9]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64,size=(255,255),alpha=3):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y
        self.size=size
        self.alpha=alpha

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, idx):
        
        _imgs=self.X[idx * self.batch_size:(idx + 1) * self.batch_size,:,:]
        
            
        
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img,size=self.size,training=self.training))
         
        
        
        imgs=np.asarray(imgs)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[idx * self.batch_size:(idx + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
    
        #mix up
        if self.training :
            r = np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            ratio=np.random.beta(self.alpha,self.alpha,imgs.shape[0])
            ratio[ratio>1]=1
            ratio[ratio<0]=0
            imgs=np.tile(ratio,(3,*size,1)).T*imgs+np.tile((1-ratio),(3,*size,1)).T*imgs2
            grapheme=np.tile(ratio,(168,1)).T*grapheme+np.tile((1-ratio),(168,1)).T*grapheme2
            vowel=np.tile(ratio,(11,1)).T*vowel+np.tile((1-ratio),(11,1)).T*vowel2
            consonant=np.tile(ratio,(7,1)).T*consonant+np.tile((1-ratio),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
   
        if self.training:
            #imgs = [randomErase(img) for img in imgs]
            pass
            
            
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y



In [10]:
def getMultiBased():
    model =  DenseNet169(weights=None, include_top=False, input_shape=(*size, 3))
    x = model.output  
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    #x = Dense(1024, activation="relu")(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236), y, train_size=0.9, random_state=8000)
del df

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=20)
checkpoint = ModelCheckpoint(filepath="tmp-dense-epoch{epoch:04}.h5")

In [13]:
batch_size=128
train_gen = DataLoader(X_train, y_train, training=True, batch_size=batch_size, size=size)
valid_gen = DataLoader(X_test, y_test, training=False, batch_size=batch_size, size=size)
#strategy = tf.distribute.MirroredStrategy()
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
#with strategy.scope():
model = getMultiBased()
#model.summary()
model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[0.5,0.25,0.25])
print("compiled")
model.fit_generator(train_gen, validation_data=valid_gen, epochs=80, callbacks=[checkpoint], workers=64, use_multiprocessing=True)
model.save("multiDenseNet.h5")

compiled
Epoch 1/80
1413/1413 [==============================] - 1148s 813ms/step - loss: 2.7003 - dense_1_loss: 4.1225 - dense_2_loss: 1.5640 - dense_3_loss: 0.9922 - dense_1_acc: 0.1529 - dense_2_acc: 0.5780 - dense_3_acc: 0.6910 - val_loss: 1.4486 - val_dense_1_loss: 2.3870 - val_dense_2_loss: 0.8221 - val_dense_3_loss: 0.5273 - val_dense_1_acc: 0.3761 - val_dense_2_acc: 0.7180 - val_dense_3_acc: 0.8047
Epoch 2/80
1413/1413 [==============================] - 1016s 719ms/step - loss: 2.0198 - dense_1_loss: 3.0186 - dense_2_loss: 1.2735 - dense_3_loss: 0.7683 - dense_1_acc: 0.5242 - dense_2_acc: 0.7317 - dense_3_acc: 0.8048 - val_loss: 0.6963 - val_dense_1_loss: 1.0353 - val_dense_2_loss: 0.3402 - val_dense_3_loss: 0.2664 - val_dense_1_acc: 0.7652 - val_dense_2_acc: 0.9311 - val_dense_3_acc: 0.9294
Epoch 3/80
1413/1413 [==============================] - 1018s 720ms/step - loss: 1.7747 - dense_1_loss: 2.6183 - dense_2_loss: 1.1582 - dense_3_loss: 0.7041 - dense_1_acc: 0.6395 - dense_2_

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1413/1413 [==============================] - 1017s 720ms/step - loss: 1.3492 - dense_1_loss: 1.9194 - dense_2_loss: 0.9535 - dense_3_loss: 0.6044 - dense_1_acc: 0.6768 - dense_2_acc: 0.7502 - dense_3_acc: 0.8281 - val_loss: 0.2882 - val_dense_1_loss: 0.3922 - val_dense_2_loss: 0.1722 - val_dense_3_loss: 0.1450 - val_dense_1_acc: 0.8981 - val_dense_2_acc: 0.9602 - val_dense_3_acc: 0.9620
Epoch 8/80
1413/1413 [==============================] - 1019s 721ms/step - loss: 1.2721 - dense_1_loss: 1.7916 - dense_2_loss: 0.9172 - dense_3_loss: 0.5879 - dense_1_acc: 0.6806 - dense_2_acc: 0.7545 - dense_3_acc: 0.8305 - val_loss: 0.2623 - val_dense_1_loss: 0.3634 - val_dense_2_loss: 0.1157 - val_dense_3_loss: 0.1126 - val_dense_1_acc: 0.9087 - val_dense_2_acc: 0.9758 - val_dense_3_acc: 0.9737
Epoch 9/80
1413/1413 [==============================] - 1018s 720ms/step - loss: 1.2007 - dense_1_loss: 1.6747 - dense_2_loss: 0.8830 - dense_3_loss: 0.5702 - dense_1_acc: 0.6848 - dense_2_acc: 0.7634 - dense_

1413/1413 [==============================] - 1019s 721ms/step - loss: 0.7880 - dense_1_loss: 1.0083 - dense_2_loss: 0.6727 - dense_3_loss: 0.4628 - dense_1_acc: 0.7703 - dense_2_acc: 0.8749 - dense_3_acc: 0.8970 - val_loss: 0.1239 - val_dense_1_loss: 0.2119 - val_dense_2_loss: 0.0619 - val_dense_3_loss: 0.0631 - val_dense_1_acc: 0.9431 - val_dense_2_acc: 0.9836 - val_dense_3_acc: 0.9823
Epoch 28/80
1413/1413 [==============================] - 1018s 721ms/step - loss: 0.7810 - dense_1_loss: 0.9988 - dense_2_loss: 0.6679 - dense_3_loss: 0.4583 - dense_1_acc: 0.7723 - dense_2_acc: 0.8791 - dense_3_acc: 0.9010 - val_loss: 0.1765 - val_dense_1_loss: 0.2151 - val_dense_2_loss: 0.0993 - val_dense_3_loss: 0.0684 - val_dense_1_acc: 0.9403 - val_dense_2_acc: 0.9739 - val_dense_3_acc: 0.9801
Epoch 29/80
1413/1413 [==============================] - 1019s 721ms/step - loss: 0.7700 - dense_1_loss: 0.9808 - dense_2_loss: 0.6632 - dense_3_loss: 0.4550 - dense_1_acc: 0.7750 - dense_2_acc: 0.8823 - dens

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1413/1413 [==============================] - 1022s 723ms/step - loss: 0.7010 - dense_1_loss: 0.8715 - dense_2_loss: 0.6300 - dense_3_loss: 0.4311 - dense_1_acc: 0.8135 - dense_2_acc: 0.9076 - dense_3_acc: 0.9232 - val_loss: 0.0829 - val_dense_1_loss: 0.1834 - val_dense_2_loss: 0.0549 - val_dense_3_loss: 0.0507 - val_dense_1_acc: 0.9512 - val_dense_2_acc: 0.9866 - val_dense_3_acc: 0.9875
Epoch 42/80
1413/1413 [==============================] - 1021s 723ms/step - loss: 0.6988 - dense_1_loss: 0.8683 - dense_2_loss: 0.6289 - dense_3_loss: 0.4298 - dense_1_acc: 0.8150 - dense_2_acc: 0.9068 - dense_3_acc: 0.9221 - val_loss: 0.1195 - val_dense_1_loss: 0.1793 - val_dense_2_loss: 0.0608 - val_dense_3_loss: 0.0575 - val_dense_1_acc: 0.9526 - val_dense_2_acc: 0.9847 - val_dense_3_acc: 0.9827
Epoch 43/80
1413/1413 [==============================] - 1022s 724ms/step - loss: 0.6920 - dense_1_loss: 0.8579 - dense_2_loss: 0.6256 - dense_3_loss: 0.4265 - dense_1_acc: 0.8171 - dense_2_acc: 0.9088 - dens

1413/1413 [==============================] - 1022s 723ms/step - loss: 0.6386 - dense_1_loss: 0.7744 - dense_2_loss: 0.5993 - dense_3_loss: 0.4062 - dense_1_acc: 0.8543 - dense_2_acc: 0.9276 - dense_3_acc: 0.9397 - val_loss: 0.1157 - val_dense_1_loss: 0.2028 - val_dense_2_loss: 0.0561 - val_dense_3_loss: 0.0527 - val_dense_1_acc: 0.9545 - val_dense_2_acc: 0.9870 - val_dense_3_acc: 0.9867
Epoch 62/80
1413/1413 [==============================] - 1021s 722ms/step - loss: 0.6381 - dense_1_loss: 0.7737 - dense_2_loss: 0.5981 - dense_3_loss: 0.4069 - dense_1_acc: 0.8528 - dense_2_acc: 0.9274 - dense_3_acc: 0.9377 - val_loss: 0.1153 - val_dense_1_loss: 0.1979 - val_dense_2_loss: 0.0533 - val_dense_3_loss: 0.0484 - val_dense_1_acc: 0.9546 - val_dense_2_acc: 0.9884 - val_dense_3_acc: 0.9879
Epoch 63/80
1170/1413 [=======================>......] - ETA: 2:46 - loss: 0.6321 - dense_1_loss: 0.7639 - dense_2_loss: 0.5951 - dense_3_loss: 0.4055 - dense_1_acc: 0.8563 - dense_2_acc: 0.9295 - dense_3_acc

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1413/1413 [==============================] - 1021s 723ms/step - loss: 0.6281 - dense_1_loss: 0.7580 - dense_2_loss: 0.5939 - dense_3_loss: 0.4024 - dense_1_acc: 0.8609 - dense_2_acc: 0.9306 - dense_3_acc: 0.9417 - val_loss: 0.1603 - val_dense_1_loss: 0.2360 - val_dense_2_loss: 0.0590 - val_dense_3_loss: 0.0583 - val_dense_1_acc: 0.9534 - val_dense_2_acc: 0.9870 - val_dense_3_acc: 0.9874
Epoch 66/80
1413/1413 [==============================] - 1023s 724ms/step - loss: 0.6263 - dense_1_loss: 0.7562 - dense_2_loss: 0.5920 - dense_3_loss: 0.4007 - dense_1_acc: 0.8629 - dense_2_acc: 0.9310 - dense_3_acc: 0.9430 - val_loss: 0.1987 - val_dense_1_loss: 0.2632 - val_dense_2_loss: 0.0667 - val_dense_3_loss: 0.0642 - val_dense_1_acc: 0.9496 - val_dense_2_acc: 0.9880 - val_dense_3_acc: 0.9870
Epoch 67/80
1413/1413 [==============================] - 1024s 724ms/step - loss: 0.6254 - dense_1_loss: 0.7538 - dense_2_loss: 0.5919 - dense_3_loss: 0.4020 - dense_1_acc: 0.8640 - dense_2_acc: 0.9315 - dens